In [23]:
import cv2
import numpy as np
import math
import pytesseract

img = cv2.imread('Sem 2.png')
dilated = cv2.dilate(img, kernel=np.ones((5, 5), np.uint8), iterations=1)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 200, 200, apertureSize=3)
cv2.imwrite("dilated.jpg", dilated)
cv2.imwrite("gray.jpg", gray)
cv2.imwrite("edges.jpg", edges)

True

In [24]:
class Line:
    HORIZONTAL = 0
    VERTICAL = 1

    def __init__(self, x0, y0, x1, y1):
        if x0 < x1:
            self.x0 = x0
            self.y0 = y0
            self.x1 = x1
            self.y1 = y1
        else:
            self.x0 = x1
            self.y0 = y1
            self.x1 = x0
            self.y1 = y0
    
    def configuration(self):
        return Line.HORIZONTAL if  abs(self.x0-self.x1) < abs(self.y0-self.y1) else Line.VERTICAL

    def __str__(self):
        return "(" + str(self.x0) + ", " + str(self.y0) + ") (" + str(self.x1) + ", " + str(self.y1) + ")"
    
    def length(self):
        return math.sqrt((self.x0-self.x1) ** 2 + (self.y0-self.y1) ** 2)

    def coordinates(self):
      return ((self.x0, self.y0), (self.x1, self.y1))

    def coordinates_as_int(self):
      return ((int(self.x0), int(self.y0)), (int(self.x1), int(self.y1)))

In [36]:
lsd = cv2.createLineSegmentDetector(cv2.LSD_REFINE_ADV, sigma_scale=0.35)
dlines = lsd.detect(edges)
lines = [Line(x0, y0, x1, y1) for x0, y0, x1, y1 in dlines[0][:, 0]]
imgCopy = img.copy()
for dline in dlines[0]:
    x0 = int(round(dline[0][0]))
    y0 = int(round(dline[0][1]))
    x1 = int(round(dline[0][2]))
    y1 = int(round(dline[0][3]))
    cv2.putText(imgCopy, "(" + str(x0) + ", " + str(y0) + ")", (x0, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    cv2.putText(imgCopy, "(" + str(x1) + ", " + str(y1) + ")", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    cv2.line(imgCopy, (x0, y0), (x1, y1), (0, 255, 0), 2)

cv2.imwrite("lsd.jpg", imgCopy)

True

In [37]:
tableStructure = np.zeros(img.shape, dtype=np.uint8)
print(img.shape)

for line in lines:
  coordinates = line.coordinates_as_int()
  cv2.line(tableStructure, coordinates[0], coordinates[1], (255, 255, 255), 1)

(322, 1680, 3)


In [38]:
kernel = np.ones((5, 5), np.uint8)
tableStructure = cv2.dilate(tableStructure, kernel, iterations=1)
cv2.imwrite("tableStructure.jpg", tableStructure)

True

In [45]:
class Rectangle:
  def __init__(self, x, y, w, h):
    self.x = x
    self.y = y
    self.w = w
    self.h = h

  def contains(self, rect):
    return (
        self.x <= rect.x and
        self.x + self.w >= rect.x + rect.w and
        self.y <= rect.y and
        self.y + self.h >= rect.y + rect.h
    )

  def is_contained(self, rect):
    return rect.contains(self)

  def get_text(self, img):
    sub_img = img[self.y:self.y+self.h, self.x:self.x+self.w]
    return pytesseract.image_to_string(sub_img, lang='eng', config='--oem 3 --psm 6').strip()
  
  def get_centre(self):
    return (self.x + self.w // 2, self.y + self.h // 2)
  
  def angle_line_centres(self, rect):
    x1, y1 = self.get_centre()
    x2, y2 = rect.get_centre()
    return math.atan2(abs(y2 - y1), abs(x2 - x1))

In [46]:
def print_rectangles(rectangles, img):
  temp = np.zeros(img.shape, dtype=np.uint8)
  temp = cv2.cvtColor(temp, cv2.COLOR_GRAY2BGR)
  for rect in rectangles:
    cv2.rectangle(temp, (rect.x, rect.y), (rect.x + rect.w, rect.y + rect.h), (0, 255, 0), 1)
  cv2.imwrite("rectangles.jpg", temp)

In [47]:
table_structure = cv2.cvtColor(tableStructure, cv2.COLOR_RGB2GRAY)
contours, hierarchy = cv2.findContours(table_structure, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
count = 0

rectangles = [] 

for c in contours:
  x, y, w, h = cv2.boundingRect(c)
  rectangles.append(Rectangle(x, y, w, h))

print_rectangles(rectangles, table_structure)

In [48]:
from PIL import Image
fcont = cv2.boundingRect(contours[-13])
x, y, w, h = fcont
print(x, y, h, w)

newImg = gray[y:y+h, x:x+w]
print(newImg.shape)
print(pytesseract.image_to_string(newImg))
Image.fromarray(newImg)


1424 17 14 5
(14, 5)



In [49]:
# remove the rectangle which contains all the other rectangles
max_x_rect = 0
min_x_rect = 0
max_y_rect = 0
min_y_rect = 0
for i in range(len(rectangles)):
  rect = rectangles[i]
  x, y, w, h = rect.x, rect.y, rect.w, rect.h
  if x > max_x_rect: max_x_rect = i
  if x < min_x_rect: min_x_rect = i
  if y > max_y_rect: max_y_rect = i
  if y < min_y_rect: min_y_rect = i
set_rect = set([max_x_rect, min_x_rect, max_y_rect, min_y_rect])
valid = [True] * len(rectangles)
for r in set_rect:
  contains_all = True
  for i in range(len(rectangles)):
    if i == r: continue
    if not rectangles[r].contains(rectangles[i]):
      contains_all = False
      break
  if contains_all:
    valid[r] = False

# Remove the rectangles which are contained by other rectangles
for i in range(len(rectangles)):
  if not valid[i]: continue
  for j in range(len(rectangles)):
    if i == j: continue
    if rectangles[i].contains(rectangles[j]):
      valid[j] = False

filtered_rect = [rectangles[i] for i in range(len(rectangles)) if valid[i]]

print_rectangles(filtered_rect, table_structure)


In [ ]:
# By now, all the rectangles are the ones which we care about
# the rectangles need not be always ordered but they are sorted by their y coordinates first and then their x coordinates

In [62]:
rows = []
row = []
rows_text = []
back = -1
for rec in filtered_rect:
    if back == -1:
        row.append(rec)
        back = rec.y
    elif abs(back - rec.y) < img.shape[0] // 20:
        row.append(rec)
        back = rec.y
    else:
        rows.append(row)
        row = [rec]
        back = -1

if len(row): rows.append(row)
rows.reverse()
for row in rows:
    row.sort(key=lambda x: x.x)


    

In [63]:
import pandas as pd

In [ ]:
# row[0] is the first row: presents the columns
columns = rows[0]

def get_maximum_intersecting_column(row, rect):
    max_intersect = -1
    max_intersect_i = 0
    for i in range(len(row)):
        r = row[i]
        intersect = 0
        x11, x12, x21, x22 = r.x, r.x + r.w, rect.x, rect.x + rect.w
        if x12 > x21 and x22 > x11: intersect = min(x12, x22) - max(x11, x21)
        if intersect > max_intersect:
            max_intersect = intersect
            max_intersect_i = i
    return max_intersect_i

df = [[] for _ in range(len(rows)-1)]
for col in columns:
    for i in range(1, len(rows)):
        row = rows[i]
        intersecting_column = get_maximum_intersecting_column(row, col)
        df[i-1].append(row[intersecting_column].get_text(gray))


pd.DataFrame(df, columns=[col.get_text(gray) for col in columns])